## Credit Curve Bootstrapper

The goal of the project is to build a bootstrapping procedure whose output will be a credit curve (an instance of the Python class CreditCurve). The input parameters to derive this curve are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotations)
* The par rate of a set of Credit Default Swap (i.e. their market quotations)

The resulting class will be used to compute the market value of a set of custom CDS.

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for CDS evaluation.

### Notes and hints

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for evaluation of risky flows.

All maturities are expressed in months and rates are expressed as fractions of one (i.e. 0.01 means 1%).

Remember to reuse the code we developed during the lessons in `finmarkets.py` as much as possible, instead of trying to rewrite everything from scratch!

In [2]:
from datetime import date
import pandas as pd

pricing_date = date(2019, 10, 31)
cds_recovery = 0.4

cds_quotes = pd.read_csv("cds_quotes_proj2.csv").to_dict('records')
ois_quotes = pd.read_csv("ois_quotes.csv").to_dict('records')
cds_to_price = pd.read_csv("cds_to_price_proj2.csv").to_dict('records')

In [3]:
from finmarkets import DiscountCurve, OvernightIndexSwap, generate_swap_dates

pillar_dates = [pricing_date]

swaps = []

for quote in ois_quotes:
    swap = OvernightIndexSwap(
        1,
        generate_swap_dates(
            pricing_date,
            quote['maturity']
        ),
        quote['rate'])
    
    swaps.append(swap)
    pillar_dates.append(swap.payment_dates[-1])
    
pillar_dates = sorted(pillar_dates)

In [7]:
import numpy as np

def objective_function(x):
    x = np.insert(x, 0, 1)
    curve = DiscountCurve(
        pricing_date, pillar_dates, x)
    
    sum_sq = 0.0
    
    for swap in swaps:
        sum_sq += swap.npv(curve) ** 2
        
    return sum_sq

In [8]:
from scipy.optimize import minimize

x0 = [1.0 for i in range(len(pillar_dates)-1)]
bounds = [(0.01, 100.0) for i in range(len(pillar_dates)-1)]

result = minimize(objective_function, x0, bounds=bounds)

In [9]:
print (result.x)
print (objective_function(x0))
print (objective_function(result.x))

[0.99991167 0.99980687 0.99970619 0.9996069  0.9994978  0.99938865
 0.99927868 0.99916369 0.99902672 0.99889136 0.99874532 0.99859865
 0.99806916 0.99748221 0.99674119 0.99582841 0.98958266 0.97817133
 0.96221328 0.94242723 0.91997882 0.89555375 0.86964684 0.84279843
 0.81599789 0.78928554 0.71346983 0.61195507 0.53245445 0.46683059]
1.399018802873446
3.816972749241944e-13


In [10]:
x = np.insert(result.x, 0, 1)
discount_curve = DiscountCurve(pricing_date, pillar_dates, x)

In [11]:
from finmarkets import generate_swap_dates, CreditDefaultSwap, CreditCurve

cds_pillar_dates = [pricing_date]

creditdefaultswaps = []
for quote in cds_quotes:
    creditdefswap = CreditDefaultSwap(
        1,
        pricing_date,
        quote['maturity']//12,
        quote['spread'])
    
    creditdefaultswaps.append(creditdefswap)
    cds_pillar_dates.append(creditdefswap.payment_dates[-1])
    
cds_pillar_dates = sorted(cds_pillar_dates)

In [15]:
def obj_function(unknown_ndps):
    unknown_ndps = np.insert(unknown_ndps, 0, 1)
    curve_c = CreditCurve(cds_pillar_dates, unknown_ndps)
    sum_sq = 0.0

    for creditdefswap in creditdefaultswaps:
        sum_sq += creditdefswap.npv(discount_curve, curve_c) ** 2

    return sum_sq

In [16]:
x0_guess = [0.5 for i in range(len(cds_pillar_dates)-1)]
bounds_credit_curve = [(0.01, 1) for i in range(len(cds_pillar_dates)-1)]

results = minimize(obj_function, x0_guess, bounds=bounds_credit_curve)

In [17]:
print (results)
print (obj_function(x0_guess))
print (obj_function(results.x))

      fun: 1.5338777486694364e-11
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.12875054e-06, -1.88231333e-06, -6.86550407e-07,  1.61817556e-06,
        2.53642663e-06, -2.50533329e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 70
      nit: 9
     njev: 10
   status: 0
  success: True
        x: array([0.90682797, 0.80418272, 0.70888456, 0.54481582, 0.29725479,
       0.08663325])
0.2718075256173687
1.5338777486694364e-11


In [24]:
x = np.insert(results.x, 0, 1)
credit_curve = CreditCurve(cds_pillar_dates, x)

In [25]:
npv_cds_to_price = []

for quote in cds_to_price:
    creditdefswapprice = CreditDefaultSwap(
        quote['nominal'],
        pricing_date,
        quote['maturity'],
        quote['spread'])
    
    npv_cds_to_price.append(creditdefswapprice.npv(discount_curve, credit_curve))

print ("Prices of CDS " + str(["{:.2f}".format(p) for p in npv_cds_to_price]))

Prices of CDS ['-214314.29', '-238050.54', '-402018.35', '-450550.01', '-596145.00', '-644676.66', '-790271.65', '-1081461.62', '-1275588.27', '-1469714.92', '-1663841.57', '-2052094.87', '-2505057.05', '-2893310.35', '-3313918.09']
